In [1]:
import pandas as pd
import numpy as np
import datetime

# чтобы видеть все столбцы
pd.set_option('display.max_columns', 100)

In [2]:
folder='2022.09.07/'

# Названия столбцов для ЖЗУ
col_names = ['№ пп','№ убытка','Дата страхового случая','Дата заявления','Страхователь',
'Номер договора','Начало действия договора','Конец действия договора','вид СС',
'тип СС','Город страхового события','Марка ТС','Модель ТС','Выезд авар. комиссара',
'Эвакуация','Независимый эксперт','Наименование СТОА','Страховая сумма по риску',
'Сумма заявленного убытка','Франшиза','Регресс','Пострадавшее ТС',
'Пострадавшее иное имущество','Дата закрытия убытка','Линия бизнеса','Суброгация',
'ПВУ','РЗУ на дату заявления','РЗУ текущеее','Дата отказа','Причина отказа','Дата оплаты',
'Сумма оплаты']


# Даты для ЖЗУ
date_columns = ['Дата страхового случая','Дата заявления','Начало действия договора',
                'Конец действия договора','Дата закрытия убытка','Дата отказа','Дата оплаты']

In [4]:
xl = pd.ExcelFile(folder+'новый ЖУ.xls')

zhzu = pd.DataFrame()

for sheet in xl.sheet_names:
    zhzu = pd.concat([zhzu, pd.read_excel(xl, sheet, header=None,
                                          names=col_names,skiprows=1)])
    
zhzu = zhzu[:-1]
zhzu=zhzu.drop_duplicates(subset=['№ убытка'])

FileNotFoundError: [Errno 2] No such file or directory: '2022.09.07/новый ЖУ.xls'

In [5]:
for col in date_columns:    
    zhzu[col]=zhzu[col].astype(object).where(zhzu[col].notnull(), None)
    zhzu[col]=pd.to_datetime(zhzu[col])

In [6]:
shifr=pd.read_excel('классификаторы шифров.xlsx',dtype={'шифр': float})

In [7]:
zhzu['шифр']=zhzu['Номер договора'].astype(str).str[:3].astype(float)
zhzu=zhzu.merge(shifr,how='left',on='шифр')

zhzu['год заявления']=zhzu['Дата заявления'].dt.year
zhzu['месяц заявления']=zhzu['Дата заявления'].dt.month

In [8]:
zhzu['неделя года']=zhzu['Дата заявления'].apply(lambda _: datetime.datetime.strftime(_,'%W'))

zhzu['статус']=np.where(zhzu['Дата оплаты'].notna(),
                        np.where(zhzu['РЗУ текущеее']>0,'Частично выпл','выпл'),
                        np.where(zhzu['РЗУ текущеее']>0,'открыто',
                                 np.where(zhzu['Дата отказа'].notna(),'отказ','закрыто')))

6

In [9]:
zhzu['Линия бизнеса_full']=np.where(zhzu['Линия бизнеса_y']!='02 - ОСАГО',
                                  zhzu['Линия бизнеса_y'],np.where(zhzu['ПВУ'].isna(),'02 - ОСАГО - Традиция',
                                                                '02 - ОСАГО'+' - '+zhzu['ПВУ']))

zhzu=zhzu[zhzu['год заявления']>2020]
pivot_count_month=zhzu.pivot_table('№ убытка', columns=['Линия бизнеса_full','статус'],
                             index=['год заявления','месяц заявления'],
                             aggfunc='count')
pivot_sum_rzu_tek_month=zhzu.pivot_table('РЗУ текущеее', columns=['Линия бизнеса_full','статус'],
                             index=['год заявления','месяц заявления'],
                             aggfunc='sum')
pivot_sum_rzu_month=zhzu.pivot_table('РЗУ на дату заявления', columns=['Линия бизнеса_full','статус'],
                             index=['год заявления','месяц заявления'],
                             aggfunc='sum')
pivot_sum_oplata_month=zhzu.pivot_table('Сумма оплаты', columns=['Линия бизнеса_full','статус'],
                             index=['год заявления','месяц заявления'],
                             aggfunc='sum')

In [10]:
pivot_count_month

Линия бизнеса_full               01 - КАСКО                                \
статус                        Частично выпл    выпл закрыто отказ открыто   
год заявления месяц заявления                                               
2021          1                         NaN   607.0     7.0  12.0    23.0   
              2                         1.0   741.0    12.0  32.0    37.0   
              3                         NaN   920.0    16.0  31.0    34.0   
              4                         NaN   905.0    16.0  33.0    51.0   
              5                         1.0   846.0    19.0  26.0    52.0   
              6                         1.0   943.0    13.0  37.0    70.0   
              7                         2.0  1021.0    13.0  35.0    74.0   
              8                         1.0  1042.0     8.0  26.0    91.0   
              9                         1.0  1112.0    12.0  43.0   104.0   
              10                        5.0  1092.0    10.0  42.0    89.0   
              11                        2.0  1015.0     9.0  44.0    94.0   
              12                        4.0  1225.0    13.0  33.0   158.0   
2022          1                         9.0  1015.0    15.0  32.0   142.0   
              2                        10.0  1069.0    16.0  42.0   239.0   
              3                        16.0  1502.0    14.0  67.0   514.0   
              4                         6.0  1115.0    18.0  79.0   560.0   
              5                         7.0   786.0    10.0  63.0   563.0   
              6                         4.0   581.0    16.0  59.0   890.0   
              7                         5.0   270.0    18.0  63.0  1138.0   
              8                         1.0    34.0    21.0  59.0  1311.0   
              9                         NaN     NaN     NaN   NaN     7.0   

Линия бизнеса_full            02 - ОСАГО - ПВУ ответсвенное                 \
статус                                                 выпл закрыто  отказ   
год заявления месяц заявления                                                
2021          1                                       730.0     NaN   61.0   
              2                                       843.0     NaN   64.0   
              3                                      1174.0     NaN   88.0   
              4                                       866.0     NaN   94.0   
              5                                       679.0     NaN   41.0   
              6                                       822.0     1.0   68.0   
              7                                       821.0     2.0   64.0   
              8                                       832.0     NaN   63.0   
              9                                      1015.0     1.0   72.0   
              10                                     1031.0     1.0   71.0   
              11                                     1022.0     NaN   94.0   
              12                                     1190.0     NaN  102.0   
2022          1                                      1220.0     NaN  108.0   
              2                                      1219.0     1.0   91.0   
              3                                      1380.0     NaN  123.0   
              4                                      1107.0     1.0  105.0   
              5                                       883.0     1.0   63.0   
              6                                       835.0     1.0  102.0   
              7                                       559.0     2.0   83.0   
              8                                       156.0     NaN   87.0   
              9                                         NaN     NaN    2.0   

Линия бизнеса_full                    02 - ОСАГО - ПВУ прямое                  \
статус                        открыто           Частично выпл    выпл закрыто   
год заявления месяц заявления                                                   
2021          1                   8

In [11]:
pivot_sum_rzu_tek_month

Линия бизнеса_full               01 - КАСКО                          \
статус                        Частично выпл      выпл закрыто отказ   
год заявления месяц заявления                                         
2021          1                         NaN      0.00     0.0   0.0   
              2                    25972.05      0.00     0.0   0.0   
              3                         NaN      0.00     0.0   0.0   
              4                         NaN      0.00     0.0   0.0   
              5                    23515.00      0.00     0.0   0.0   
              6                    39100.16      0.00     0.0   0.0   
              7                    22450.75      0.00     0.0   0.0   
              8                   279690.00      0.00     0.0   0.0   
              9                      154.00      0.00     0.0   0.0   
              10                  362848.97      0.00     0.0   0.0   
              11                  250455.07      0.00     0.0   0.0   
              12                  466557.16 -68910.97     0.0   0.0   
2022          1                   438051.84      0.00     0.0   0.0   
              2                   936169.68      0.00     0.0   0.0   
              3                  3621694.40      0.00     0.0   0.0   
              4                   479623.28      0.00     0.0   0.0   
              5                   810379.76      0.00     0.0   0.0   
              6                  1336377.09      0.00     0.0   0.0   
              7                  3151579.59      0.00     0.0   0.0   
              8                   728263.40      0.00     0.0   0.0   
              9                         NaN       NaN     NaN   NaN   

Линия бизнеса_full                          02 - ОСАГО - ПВУ ответсвенное  \
статус                              открыто                          выпл   
год заявления месяц заявления                                               
2021          1                3.772114e+06                           0.0   
              2                3.347788e+06                           0.0   
              3                1.840244e+06                           0.0   
              4                3.452455e+06                           0.0   
              5                4.126566e+06                           0.0   
              6                1.148956e+07                           0.0   
              7                5.480300e+06                           0.0   
              8                9.419341e+06                           0.0   
              9                1.139781e+07                           0.0   
              10               1.761086e+07                           0.0   
              11               3.028919e+07                           0.0   
              12               3.441458e+07                           0.0   
2022          1                3.987113e+07                           0.0   
              2                5.000519e+07                           0.0   
              3                8.033833e+07                           0.0   
              4                9.502767e+07                           0.0   
              5                9.473471e+07                           0.0   
              6                1.682403e+08                           0.0   
              7                1.733638e+08                           0.0   
              8                1.696587e+08                           0.0   
              9                9.390000e+05                           NaN   

Линия бизнеса_full                                           \
статус                        закрыто    отказ      открыто   
год заявления месяц заявления                                 
2021          1                   NaN      0.0    534679.00   
              2                   NaN      0.0    560528.00   
              3                   NaN      0.0   1174703.00   
              4                   NaN      0.0    640990.00   
     

In [9]:
with pd.ExcelWriter(folder+'_out_данные.xlsx') as writer:  
    pivot_count_month.to_excel(writer,sheet_name='кол-во мес')
    pivot_sum_rzu_tek_month.to_excel(writer,sheet_name='РЗУ тек мес')
    pivot_sum_rzu_month.to_excel(writer,sheet_name='РЗУ заявл мес')
    pivot_sum_oplata_month.to_excel(writer,sheet_name='Оплачено нед')

zhzu.to_excel(folder+'_zhzu_out.xlsx')

In [10]:
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as ipy
from ipywidgets import Output, VBox, widgets

In [11]:
df_count=pivot_count_month
df_now=pivot_sum_rzu_tek_month
df_declared=pivot_sum_rzu_month
df_paid=pivot_sum_oplata_month

In [12]:
def b_line_part_maker(df_full, b_line):
    #Получение строк по линии бизнеса
    df_temp=df_full[b_line]
    #Сохранение периодов
    df_temp['Период'] = [str(i[0])+'/'+str(i[1]) for i in df_temp.index.values]
    #Перезапись столбцов
    df_temp=pd.DataFrame(df_temp.values, columns=df_temp.columns)
    #Создание индекса по периодам для удобной визуализации
    df_temp=df_temp.set_index('Период')
    #Перезапись значений в виде чисел
    y_labels=df_temp.columns
    for i in y_labels:
        df_temp[i]=pd.to_numeric(df_temp[i], downcast='float')
    df_temp=df_temp.fillna(0)
    return df_temp

In [13]:
df_count_CMTPL=b_line_part_maker(df_count, '01 - КАСКО')
fig_1 = go.FigureWidget(px.bar(df_count_CMTPL,y=df_count_CMTPL.columns,
                               title='Количество на месяц',
                               color_discrete_map={'закрыто': 'red','выпл': 'green','открыто': 'gold',
                                                                 'отказ': 'cornflowerblue','Частично выпл': 'cyan'},
                               category_orders={'статус':['выпл','открыто','Частично выпл','закрыто','отказ']},
                               text_auto='.2s'))

df_now_CMTPL=b_line_part_maker(df_now, '01 - КАСКО')
fig_2 = go.FigureWidget(px.bar(df_now_CMTPL,y=df_now_CMTPL.columns,
                               title='РЗУ на текущий месяц',
                               color_discrete_map={'закрыто': 'red','выпл': 'green','открыто': 'gold',
                                                                 'отказ': 'cornflowerblue','Частично выпл': 'cyan'},
                               category_orders={'статус':['выпл','открыто','Частично выпл','закрыто','отказ']},
                               text_auto='.2s'))

df_declared_CMTPL=b_line_part_maker(df_declared, '01 - КАСКО')
fig_3 = go.FigureWidget(px.bar(df_declared_CMTPL,y=df_declared_CMTPL.columns,
                               title='РЗУ на дату заявления',
                               color_discrete_map={'закрыто': 'red','выпл': 'green','открыто': 'gold',
                                                                 'отказ': 'cornflowerblue','Частично выпл': 'cyan'},
                               category_orders={'статус':['выпл','открыто','Частично выпл','закрыто','отказ']},
                               text_auto='.2s'))

df_paid_CMTPL=b_line_part_maker(df_paid, '01 - КАСКО')
fig_4 = go.FigureWidget(px.bar(df_paid_CMTPL,y=df_paid_CMTPL.columns,
                               title='оплачено на текущий момент',
                               color_discrete_map={'закрыто': 'red','выпл': 'green','открыто': 'gold',
                                                                 'отказ': 'cornflowerblue','Частично выпл': 'cyan'},
                               category_orders={'статус':['выпл','открыто','Частично выпл','закрыто','отказ']},
                               text_auto='.2s'))

varianti=list(set([i[0] for i in df_count.columns.values]))
varianti.sort()

my_filter = widgets.Dropdown(description='Filter:',value=varianti[0],
                             options=varianti)



<ipython-input-12-d1765466fd43>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Период'] = [str(i[0])+'/'+str(i[1]) for i in df_temp.index.values]
<ipython-input-12-d1765466fd43>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
def fig_update(fig_df):
    fig=fig_df[0]; df_now=fig_df[1]
    nowDF = b_line_part_maker(df_now, my_filter.value)
    nowTrace = px.bar(nowDF,y=nowDF.columns, color_discrete_map={'закрыто': 'red','выпл': 'green','открыто': 'gold',
                                                                 'отказ': 'cornflowerblue','Частично выпл': 'cyan'},
                      category_orders={'статус':['выпл','открыто','Частично выпл','закрыто','отказ']},
                      text_auto='.2s')    
    with fig.batch_update():
        fig.data = []
        for tr in nowTrace.data: fig.add_bar(x=tr.x,y=tr.y,name=tr.name,marker=tr.marker,texttemplate=tr.texttemplate)
        #marker сохраняет цвета колонок

In [15]:
def response(change):
    fig_dfs=[[fig_1,df_count],[fig_2,df_now],[fig_3,df_declared],[fig_4,df_paid]]
    for fig_df in fig_dfs: fig_update(fig_df)   

In [16]:
my_filter.observe(response, names="value")
    
container1 = widgets.HBox([my_filter]) 
    
widgets.VBox([container1,fig_1,fig_2,fig_3,fig_4])

<ipython-input-12-d1765466fd43>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-d1765466fd43>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-d1765466fd43>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-d176546